In [1]:
#Query 2 Initialisation with Dataframe
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
import time

spark = SparkSession \
    .builder \
    .appName("Query 2 Dataframe csv") \
    .getOrCreate()

window_spec = Window.partitionBy("year").orderBy(F.desc("closed_case_rate"))
start_time = time.time()
dataframe = (spark.read
              .option("header", "true")
              .option("inferSchema", "true")
              .csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_*.csv"))

dataframe = (dataframe
              .withColumn("report_date", F.to_timestamp("Date Rptd", "MM/dd/yyyy hh:mm:ss a"))
              .withColumn("year", F.year("report_date"))
              .withColumnRenamed("AREA NAME", "precinct"))

closed_case_rate_df = (dataframe
                       .groupBy("year", "precinct")
                       .agg(((F.sum(F.when(F.col("Status") != "IC", 1).otherwise(0)) / F.count("*")) * 100)
                       .alias("closed_case_rate")))


ranked_precincts = (closed_case_rate_df
                    .withColumn("#", F.row_number().over(window_spec))
                    .filter(F.col("#") <= 3))

ranked_precincts.show(50, truncate=False)

end_time = time.time()
print(f"Time taken: {end_time - start_time:.2f} seconds")


Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
4123,application_1732639283265_4063,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----------+------------------+---+
|year|precinct   |closed_case_rate  |#  |
+----+-----------+------------------+---+
|2010|Rampart    |32.947355855318136|1  |
|2010|Olympic    |31.962706191728422|2  |
|2010|Harbor     |29.63203463203463 |3  |
|2011|Olympic    |35.212167689161554|1  |
|2011|Rampart    |32.511779630300836|2  |
|2011|Harbor     |28.652205202015008|3  |
|2012|Olympic    |34.414818310523835|1  |
|2012|Rampart    |32.9464181029429  |2  |
|2012|Harbor     |29.815133276010318|3  |
|2013|Olympic    |33.52812271731191 |1  |
|2013|Rampart    |32.08287360549221 |2  |
|2013|Harbor     |29.16422459266206 |3  |
|2014|Van Nuys   |31.80567315834039 |1  |
|2014|West Valley|31.31198995605775 |2  |
|2014|Mission    |31.16279069767442 |3  |
|2015|Van Nuys   |32.64134698172773 |1  |
|2015|West Valley|30.27597402597403 |2  |
|2015|Mission    |30.179460678380153|3  |
|2016|Van Nuys   |31.880755720117726|1  |
|2016|West Valley|31.54798761609907 |2  |
|2016|Foothill   |29.8702918433524

In [2]:
#Query 2 Initialisation with SQL API
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, IntegerType, FloatType, StringType, TimestampType
from pyspark.sql.functions import col, udf
import time
import csv


sc = SparkSession \
    .builder \
    .appName("SQL API query 2 execution") \
    .getOrCreate() \
    
#Start timer
start_time = time.time()

#Read data from both datasets (10-19 and 20-present)
crime_df = spark.read.csv(
    "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_*.csv",
    header=True,
    inferSchema=True
)
crime_df.createOrReplaceTempView("crime_df")
      
query="""
    WITH initial_data AS (
        SELECT 
            SUBSTR(`Date Rptd`, 7, 4) AS year,
            `AREA NAME` AS precinct,
            Status
    FROM crime_df
),
    ranked_data AS (
        SELECT
            year,
            precinct,
            (COUNT(CASE WHEN Status != 'IC' THEN 1 END) * 100) / COUNT(*) AS closed_case_rate,
            ROW_NUMBER() OVER (PARTITION BY year ORDER BY (COUNT(CASE WHEN Status != 'IC' THEN 1 END) * 100) / COUNT(*) DESC) AS `#`
    FROM initial_data
    GROUP BY year, precinct
    )
    SELECT 
        year,
        precinct,
        closed_case_rate,
        `#`
    FROM ranked_data
    WHERE `#` < 4;
"""
results= spark.sql(query)
results.show(45)
end_time = time.time()

print(f"Time taken: {end_time-start_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----------+------------------+---+
|year|   precinct|  closed_case_rate|  #|
+----+-----------+------------------+---+
|2010|    Rampart|32.947355855318136|  1|
|2010|    Olympic|31.962706191728426|  2|
|2010|     Harbor| 29.63203463203463|  3|
|2011|    Olympic|35.212167689161554|  1|
|2011|    Rampart|32.511779630300836|  2|
|2011|     Harbor| 28.65220520201501|  3|
|2012|    Olympic|34.414818310523835|  1|
|2012|    Rampart|  32.9464181029429|  2|
|2012|     Harbor|29.815133276010318|  3|
|2013|    Olympic| 33.52812271731191|  1|
|2013|    Rampart| 32.08287360549222|  2|
|2013|     Harbor|29.164224592662055|  3|
|2014|   Van Nuys| 31.80567315834039|  1|
|2014|West Valley|31.311989956057754|  2|
|2014|    Mission|31.162790697674417|  3|
|2015|   Van Nuys|32.641346981727736|  1|
|2015|West Valley|30.275974025974026|  2|
|2015|    Mission|30.179460678380156|  3|
|2016|   Van Nuys|31.880755720117726|  1|
|2016|West Valley| 31.54798761609907|  2|
|2016|   Foothill|29.8702918433524

In [3]:
#Query 2b write parquet
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Query 2b write parquet") \
    .getOrCreate()

dataframe= spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_*.csv", header=True)

dataframe.coalesce(1).write.mode("overwrite").parquet("s3://groups-bucket-dblab-905418150721/group12/main_dataset_parquet") 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
#Query 2b Parquet Initialisation with Dataframe 
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
import time

spark = SparkSession \
    .builder \
    .appName("Query 2 Dataframe parquet") \
    .getOrCreate()

window_spec = Window.partitionBy("year").orderBy(F.desc("closed_case_rate"))
start_time = time.time()
dataframe = (spark.read
              .option("header", "true")
              .option("inferSchema", "true")
              .parquet("s3://groups-bucket-dblab-905418150721/group12/main_dataset_parquet"))

dataframe = (dataframe
              .withColumn("report_date", F.to_timestamp("Date Rptd", "MM/dd/yyyy hh:mm:ss a"))
              .withColumn("year", F.year("report_date"))
              .withColumnRenamed("AREA NAME", "precinct"))

closed_case_rate_df = (dataframe
                       .groupBy("year", "precinct")
                       .agg(((F.sum(F.when(F.col("Status") != "IC", 1).otherwise(0)) / F.count("*")) * 100)
                       .alias("closed_case_rate")))


ranked_precincts = (closed_case_rate_df
                    .withColumn("#", F.row_number().over(window_spec))
                    .filter(F.col("#") <= 3))

ranked_precincts.show(50, truncate=False)

end_time = time.time()
print(f"Time taken: {end_time - start_time:.2f} seconds")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----------+------------------+---+
|year|precinct   |closed_case_rate  |#  |
+----+-----------+------------------+---+
|2010|Rampart    |32.947355855318136|1  |
|2010|Olympic    |31.962706191728422|2  |
|2010|Harbor     |29.63203463203463 |3  |
|2011|Olympic    |35.212167689161554|1  |
|2011|Rampart    |32.511779630300836|2  |
|2011|Harbor     |28.652205202015008|3  |
|2012|Olympic    |34.414818310523835|1  |
|2012|Rampart    |32.9464181029429  |2  |
|2012|Harbor     |29.815133276010318|3  |
|2013|Olympic    |33.52812271731191 |1  |
|2013|Rampart    |32.08287360549221 |2  |
|2013|Harbor     |29.16422459266206 |3  |
|2014|Van Nuys   |31.80567315834039 |1  |
|2014|West Valley|31.31198995605775 |2  |
|2014|Mission    |31.16279069767442 |3  |
|2015|Van Nuys   |32.64134698172773 |1  |
|2015|West Valley|30.27597402597403 |2  |
|2015|Mission    |30.179460678380153|3  |
|2016|Van Nuys   |31.880755720117726|1  |
|2016|West Valley|31.54798761609907 |2  |
|2016|Foothill   |29.8702918433524